In [10]:
import os
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import Search

In [11]:
cwd = "c:\\Users\\m1865\\Desktop\\DISC"
cwd_Images_Raw = cwd + "\\Sentinel-2 Images Raw"
cwd_Images_Processed = cwd + "\\Sentinel-2 Images Processed"

In [12]:
# Site Name
df = pd.read_excel(cwd + "//Flox_sites_table_final.xlsx")
df.head()

,Number,Site code,Site name,Country,LAT,LON,Main ecosystem,FLOX Active,Homogeneity test passed,Network,Link
0,1,AT-Mmg,Mieming,Austria,47.3167,10.9702,Forest,Yes,Yes,NaN,https://www.europe-fluxdata.eu/home/site-detai...
1,2,BE-Bra,Brasschaat,Belgium,51.3077,4.5198,Forest,Yes,Yes,ICOS,https://meta.icos-cp.eu/resources/stations/ES_...
2,3,BE-WAL,BE - WAL,Belgium,50.5516,4.7461,Crop,No,No,NaN,https://essd.copernicus.org/articles/16/1283/2...
3,4,NaN,Tapajos KM67,Brasil,-2.8567,-54.9589,Forest,Planned,Yes,NaN,NaN
4,5,CANADA-ONT,Ontario,Canada,42.7102,-80.3573,Forest,Yes,Yes,NaN,NaN


In [13]:
# Unique main ecosystem
list(df['Main ecosystem'].unique())

['Forest', 'Crop', 'Grasslands', 'Wet', 'Savannah', 'Tundra']

In [14]:
gdf = gpd.GeoDataFrame(
    df[['Site name','Main ecosystem','Homogeneity test passed','LAT','LON']], 
    geometry=gpd.points_from_xy(df.LON, df.LAT), 
    crs = "EPSG:4326"
)
gdf.head()

,Site name,Main ecosystem,Homogeneity test passed,LAT,LON,geometry
0,Mieming,Forest,Yes,47.3167,10.9702,POINT (10.9702 47.3167)
1,Brasschaat,Forest,Yes,51.3077,4.5198,POINT (4.5198 51.3077)
2,BE - WAL,Crop,No,50.5516,4.7461,POINT (4.7461 50.5516)
3,Tapajos KM67,Forest,Yes,-2.8567,-54.9589,POINT (-54.9589 -2.8567)
4,Ontario,Forest,Yes,42.7102,-80.3573,POINT (-80.3573 42.7102)


In [15]:
# def getMarker(eco):
#     match eco:
#         case 'Evergreen Needleleaf Forests':
#             return "leaf"
#         case 'Crop':
#             return 'carrot'
#         case 'Forest':
#             return 'tree'
#         case 'Grasslands':
#             return 'cow'
#         case 'Wet':
#             return 'water'
#         case 'Oak':
#             return 'circle'
#         case 'Palm oil':
#             return 'bottle-droplet'
#         case 'P. halepensis':
#             return 'apple-whole'
#         case 'Grapevine':
#             return 'wine-glass'
#         case 'Evergreen Mediterranean forest':
#             return 'fish'
#         case 'Deciduous Needleleaf Forests':
#             return 'snowflake'
#         case 'Savannah':
#             return 'hourglass-end'
#         case 'Peat':
#             return 'fire'
#         case 'Mixed deciduous mountain forest':
#             return 'mountain'
#         case _:
#             return 'exclamation-triangle'

In [38]:
def getMarker(eco):
    match eco:
        case 'Forest':
            return "F"
        case 'Wet':
            return 'W'
        case "Tundra":
            return "T"
        case "Grasslands":
            return "G"
        case "Savannah":
            return "S"
        case "Crop":
            return "C"

In [39]:
def testColor(result):
    if result == 'Yes':
        return 'green'
    else:
        return 'red'

In [61]:
m = folium.Map(location=[47.316500, 10.970300], zoom_start=4, tiles=None)


tile = folium.TileLayer(
    "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}", attr='Esri', name="Esri Satellite",overlay = False
).add_to(m)

# icon1 = folium.CustomIcon(icon_image = 'https://leafletjs.com/examples/custom-icons/leaf-red.png', icon_size=(50,50))

# Build folium geojson (site point)
for i in range(df.shape[0]):
    # print(i)
    temp_Name = df['Site name'][i]
    temp_Eco = df['Main ecosystem'][i]
    temp_Result = df['Homogeneity test passed'][i]
    temp_Lat = df['LAT'][i]
    temp_Lon = df['LON'][i]
    # Set custom icon
    icon1 = folium.plugins.BeautifyIcon(
        icon="circle", 
        border_color=testColor(temp_Result), 
        border_width = 0.5,
        text_color=testColor(temp_Result), 
        icon_shape="circle", 
        number = getMarker(temp_Eco),
        icon_size = [10,10],
        innerIconAnchor = [0,-2],
        innerIconStyle = "font-size: 8px"
)
    folium.Marker(
        location = [temp_Lat,temp_Lon],
        popup = temp_Name,
        icon = icon1
        # icon = folium.Icon(icon = getMarker(temp_Eco), prefix= 'fa', color = testColor(temp_Result))
    ).add_to(m)

# # Search
# statesearch = Search(
#     layer=geo_site,
#     geom_type="Point",
#     placeholder="Search for a site",
#     collapsed=False,
#     search_label="Site name",
#     search_zoom = 15,
#     weight=3,
# ).add_to(m)

# folium.LayerControl().add_to(m)
m

In [62]:
m.save(cwd + "\\Site Map 56.html")